In [1]:
##### Change for PH decay rate


# Model V2
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple
import math
import random
import csv

import pymc as pm
import pandas as pd
import numpy as np
import arviz as az


# =====================================================================
# 1. PARAMETERS AND STATE DEFINITIONS
# =====================================================================

@dataclass
class MFCParams:
    # --- HRT control parameters (pH-driven) ---
    HRT_nom: float = 24.0
    HRT_min: float = 6.0
    HRT_max: float = 72.0
    K_low_pH: float = 4.0
    K_high_pH: float = 4.0

    pH_low: float = 6.8
    pH_high: float = 7.2
    pH_low_action: float = 6.0
    pH_high_action: float = 8.0

    # --- Time constants (1-minute dt, essentially instantaneous) ---
    tau_EC_hours: float = 0.001
    tau_pH_hours: float = 0.001
    tau_TM_hours: float = 0.001  # used for iron + other TM states

    # --- EC contribution coefficients from inputs ---
    k_EC_spirulina: float = 0.5
    k_EC_urine: float = 1.2
    k_EC_winery_ww: float = 0.8
    k_EC_iron_oxide: float = 0.7
    k_EC_molasses: float = 0.3

    EC_min: float = 0.2
    EC_max: float = 25.0

    # --- pH contribution parameters from inputs + temperature ---
    pH_base: float = 7.0
    k_pH_alkaline_urine: float = 0.4
    k_pH_alkaline_winery: float = 0.3
    k_pH_alkaline_spirulina: float = 0.1
    k_pH_acid_iron: float = 0.2
    k_pH_acid_molasses: float = 0.15

    pH_min: float = 4.5
    pH_max: float = 9.5

    k_pH_temp: float = -0.01  # pH change per °F relative to 70°F

    # --- pH decay rate ---
    pH_decay_per_24h: float = 0.4  # pH units lost per 24 hours


    #### Castro-Ramos, Job Jonathan, et al. “Effect of the Initial pH on the Anaerobic Digestion Process of Dairy Cattle Manure.” AMB Express [Berlin/Heidelberg], vol. 12, no. 1, no. 162, 2022, https://doi.org/10.1186/s13568-022-01486-8.

    # --- EC–pH coupling thresholds (phase-based) ---
    pH_acidogenic_max: float = 7.5
    pH_methanogenic_min: float = 7.5
    pH_methanogenic_max: float = 8.0
    pH_imbalance_min: float = 8.3

    # Acidogenic: EC = 3.18 * pH - 17.46
    ec_ph_acidogenic_slope: float = 3.18
    ec_ph_acidogenic_intercept: float = -17.46

    # Imbalance: EC = 3.47 * pH - 24.01
    ec_ph_imbalance_slope: float = 3.47
    ec_ph_imbalance_intercept: float = -24.01

    # --- Iron oxide target range (controlled input + state) ---
    iron_oxide_min: float = 2.0
    iron_oxide_max: float = 4.0
    iron_oxide_target: float = 3.0  # midpoint

    # --- Trace metal contribution factors from each substrate ---
    # Increased so TM effects on Voltage are non-negligible
    k_TM_spirulina: float = 0.4
    k_TM_urine: float = 0.2
    k_TM_winery_ww: float = 0.3
    k_TM_molasses: float = 0.1

    # Individual trace metal bounds (intrinsic states) – expanded ranges
    TM_spirulina_min: float = 0.0
    TM_spirulina_max: float = 5.0
    TM_urine_min: float = 0.0
    TM_urine_max: float = 3.0
    TM_winery_min: float = 0.0
    TM_winery_max: float = 4.0
    TM_molasses_min: float = 0.0
    TM_molasses_max: float = 2.0

    # --- ORP model parameters (anaerobic baseline) ---
    a0_ORP: float = -200.0      # baseline ORP (mV) for anaerobic condition
    a_EC_ORP: float = 8.0       # mV per unit EC
    a_pH_ORP: float = -40.0     # mV per (pH - 7)
    a_iron_ORP: float = 10.0    # mV per unit iron_oxide
    a_TM_other_ORP: float = 8.0 # mV per unit total other TM (non-negligible)

    # --- Voltage model parameters (both Fe and TM significant) ---
    E0: float = 0.4             # baseline open-circuit voltage (V)
    b_EC_V: float = 0.04        # V per log(EC)
    b_pH_V: float = -0.025      # V per (pH - 7)
    b_iron_V: float = 0.02      # V per unit iron_oxide
    b_TM_other_V: float = 0.03  # V per unit total other TM (stronger than Fe here)

    EC_ref_for_log: float = 1.0

    # Measurement noise (for simulation) – reduced so effects are detectable
    sigma_ORP: float = 4.0      # mV
    sigma_V: float = 0.005      # V


@dataclass
class MFCState:
    EC: float
    pH: float
    iron_oxide: float
    TM_spirulina: float
    TM_urine: float
    TM_winery: float
    TM_molasses: float

    def total_other_TM(self) -> float:
        return self.TM_spirulina + self.TM_urine + self.TM_winery + self.TM_molasses


# =====================================================================
# 2. CONTROL LAW AND EC–pH COUPLING
# =====================================================================

def hrt_control(pH: float, params: MFCParams) -> float:
    if params.pH_low <= pH <= params.pH_high:
        return params.HRT_nom

    if pH < params.pH_low_action:
        delta_pH = params.pH_low - pH
        HRT = params.HRT_nom + params.K_low_pH * delta_pH
        return max(params.HRT_min, min(params.HRT_max, HRT))

    if pH > params.pH_high_action:
        delta_pH = pH - params.pH_high
        HRT = params.HRT_nom - params.K_high_pH * delta_pH
        return max(params.HRT_min, min(params.HRT_max, HRT))

    return params.HRT_nom


def get_phase(pH: float, params: MFCParams) -> str:
    if pH < params.pH_acidogenic_max:
        return "acidogenic"
    elif params.pH_methanogenic_min <= pH <= params.pH_methanogenic_max:
        return "methanogenic"
    elif pH > params.pH_imbalance_min:
        return "imbalance"
    else:
        return "transition"


def apply_ec_ph_coupling(EC_raw: float, pH: float, params: MFCParams) -> float:
    phase = get_phase(pH, params)

    if phase == "acidogenic":
        EC_coupled = params.ec_ph_acidogenic_slope * pH + params.ec_ph_acidogenic_intercept
        EC_final = 0.7 * EC_coupled + 0.3 * EC_raw
    elif phase == "imbalance":
        EC_coupled = params.ec_ph_imbalance_slope * pH + params.ec_ph_imbalance_intercept
        EC_final = 0.7 * EC_coupled + 0.3 * EC_raw
    elif phase == "methanogenic":
        EC_final = EC_raw
    else:
        if pH > params.pH_methanogenic_max:
            blend = (pH - params.pH_methanogenic_max) / (params.pH_imbalance_min - params.pH_methanogenic_max)
            blend = max(0.0, min(1.0, blend))
            EC_imbalance = params.ec_ph_imbalance_slope * pH + params.ec_ph_imbalance_intercept
            EC_final = (1 - blend) * EC_raw + blend * (0.7 * EC_imbalance + 0.3 * EC_raw)
        else:
            EC_final = EC_raw

    return max(params.EC_min, min(params.EC_max, EC_final))


# =====================================================================
# 3. DETERMINISTIC INPUT → STATE MAPPING
# =====================================================================

def compute_state_targets(u: Dict[str, float], HRT: float, params: MFCParams) -> MFCState:
    spir = u.get('spirulina', 0.0)
    uri = u.get('urine', 0.0)
    ww = u.get('winery_ww', 0.0)
    fe = u.get('iron_oxide', 0.0)
    mol = u.get('molasses', 0.0)
    T_actual_F = u.get('T_actual_F', 70.0)

    # pH driver: inputs + temperature
    pH_target = params.pH_base
    pH_target += params.k_pH_alkaline_urine * uri
    pH_target += params.k_pH_alkaline_winery * ww
    pH_target += params.k_pH_alkaline_spirulina * spir
    pH_target -= params.k_pH_acid_iron * fe
    pH_target -= params.k_pH_acid_molasses * mol

    total_load = spir + uri + ww + fe + mol
    if total_load > 0:
        pH_target = params.pH_base + (pH_target - params.pH_base) / (1.0 + total_load)

    dT = T_actual_F - 70.0
    pH_target += params.k_pH_temp * dT
    pH_target = max(params.pH_min, min(params.pH_max, pH_target))

    # EC driver: inputs + HRT, then coupled with pH
    EC_raw = (
        params.k_EC_spirulina * spir +
        params.k_EC_urine * uri +
        params.k_EC_winery_ww * ww +
        params.k_EC_iron_oxide * fe +
        params.k_EC_molasses * mol
    )
    HRT_factor = 1.0 + 0.1 * (HRT - params.HRT_nom) / params.HRT_nom
    EC_raw *= HRT_factor

    EC_target = apply_ec_ph_coupling(EC_raw, pH_target, params)

    # Iron oxide: controlled, fixed to 2–4 g/L
    iron_target = params.iron_oxide_target + (fe - 0.5) * (params.iron_oxide_max - params.iron_oxide_min)
    iron_target = max(params.iron_oxide_min, min(params.iron_oxide_max, iron_target))

    # Intrinsic trace metals: separate states, scaled up so they matter
    TM_spirulina_target = params.k_TM_spirulina * spir
    TM_spirulina_target = max(params.TM_spirulina_min, min(params.TM_spirulina_max, TM_spirulina_target))

    TM_urine_target = params.k_TM_urine * uri
    TM_urine_target = max(params.TM_urine_min, min(params.TM_urine_max, TM_urine_target))

    TM_winery_target = params.k_TM_winery_ww * ww
    TM_winery_target = max(params.TM_winery_min, min(params.TM_winery_max, TM_winery_target))

    TM_molasses_target = params.k_TM_molasses * mol
    TM_molasses_target = max(params.TM_molasses_min, min(params.TM_molasses_max, TM_molasses_target))

    return MFCState(
        EC=EC_target,
        pH=pH_target,
        iron_oxide=iron_target,
        TM_spirulina=TM_spirulina_target,
        TM_urine=TM_urine_target,
        TM_winery=TM_winery_target,
        TM_molasses=TM_molasses_target
    )


def update_states(x: MFCState, u: Dict[str, float], dt_hours: float,
                  params: MFCParams) -> Tuple[MFCState, float]:
    HRT = hrt_control(x.pH, params)
    x_star = compute_state_targets(u, HRT, params)

    alpha_EC = min(1.0, dt_hours / params.tau_EC_hours)
    alpha_pH = min(1.0, dt_hours / params.tau_pH_hours)
    alpha_TM = min(1.0, dt_hours / params.tau_TM_hours)

    EC_next = x.EC + alpha_EC * (x_star.EC - x.EC)
    pH_next = x.pH + alpha_pH * (x_star.pH - x.pH)

    # Apply pH decay: 0.4 pH per 24 hours
    pH_decay_per_hour = params.pH_decay_per_24h / 24.0
    pH_decay_this_step = pH_decay_per_hour * dt_hours
    pH_next = pH_next - pH_decay_this_step

    iron_next = x.iron_oxide + alpha_TM * (x_star.iron_oxide - x.iron_oxide)
    TM_spir_next = x.TM_spirulina + alpha_TM * (x_star.TM_spirulina - x.TM_spirulina)
    TM_uri_next = x.TM_urine + alpha_TM * (x_star.TM_urine - x.TM_urine)
    TM_win_next = x.TM_winery + alpha_TM * (x_star.TM_winery - x.TM_winery)
    TM_mol_next = x.TM_molasses + alpha_TM * (x_star.TM_molasses - x.TM_molasses)

    EC_next = max(params.EC_min, min(params.EC_max, EC_next))
    pH_next = max(params.pH_min, min(params.pH_max, pH_next))
    iron_next = max(params.iron_oxide_min, min(params.iron_oxide_max, iron_next))
    TM_spir_next = max(params.TM_spirulina_min, min(params.TM_spirulina_max, TM_spir_next))
    TM_uri_next = max(params.TM_urine_min, min(params.TM_urine_max, TM_uri_next))
    TM_win_next = max(params.TM_winery_min, min(params.TM_winery_max, TM_win_next))
    TM_mol_next = max(params.TM_molasses_min, min(params.TM_molasses_max, TM_mol_next))

    x_next = MFCState(
        EC=EC_next,
        pH=pH_next,
        iron_oxide=iron_next,
        TM_spirulina=TM_spir_next,
        TM_urine=TM_uri_next,
        TM_winery=TM_win_next,
        TM_molasses=TM_mol_next
    )

    return x_next, HRT


# =====================================================================
# 4. OUTPUT MODEL: STATES → ORP AND VOLTAGE
# =====================================================================

def compute_outputs(x: MFCState, params: MFCParams,
                    add_noise: bool = True) -> Tuple[Tuple[float, float], Tuple[float, float]]:
    total_other_TM = x.total_other_TM()

    ORP_true = (
        params.a0_ORP +
        params.a_EC_ORP * x.EC +
        params.a_pH_ORP * (x.pH - 7.0) +
        params.a_iron_ORP * x.iron_oxide +
        params.a_TM_other_ORP * total_other_TM
    )

    EC_for_log = max(1e-6, x.EC / params.EC_ref_for_log)
    V_true = (
        params.E0 +
        params.b_EC_V * math.log(EC_for_log) +
        params.b_pH_V * (x.pH - 7.0) +
        params.b_iron_V * x.iron_oxide +
        params.b_TM_other_V * total_other_TM
    )

    if add_noise:
        ORP_obs = ORP_true + random.gauss(0.0, params.sigma_ORP)
        V_obs = V_true + random.gauss(0.0, params.sigma_V)
    else:
        ORP_obs = ORP_true
        V_obs = V_true

    return (ORP_obs, V_obs), (ORP_true, V_true)


# =====================================================================
# 5. ARDUINO-STYLE OUTPUT, SIMULATION DRIVER, CSV EXPORT, SUMMARY
# =====================================================================

def print_arduino_output(minute: int, u: Dict[str, float], x: MFCState,
                         ORP: float, V: float, HRT: float, phase: str) -> None:
    hours = minute // 60
    mins = minute % 60

    print("=" * 70)
    print(f"[{hours:02d}:{mins:02d}] MFC Serial Monitor - Minute {minute}")
    print("=" * 70)
    print("--- DOSING (this minute) ---")
    print(f"  Spirulina:   {u.get('spirulina', 0.0):.3f} units")
    print(f"  Urine:       {u.get('urine', 0.0):.3f} units")
    print(f"  Winery WW:   {u.get('winery_ww', 0.0):.3f} units")
    print(f"  Iron Oxide:  {u.get('iron_oxide', 0.0):.3f} units")
    print(f"  Molasses:    {u.get('molasses', 0.0):.3f} units")
    print(f"  Temperature: {u.get('T_actual_F', 70.0):.1f} F")
    print()
    print("--- SENSOR READINGS ---")
    print(f"  EC:          {x.EC:.2f} mS/cm")
    print(f"  pH:          {x.pH:.2f}")
    print(f"  ORP:         {ORP:.1f} mV")
    print(f"  Voltage:     {V:.4f} V")
    print()
    print("--- TRACE METALS ---")
    print(f"  Iron Oxide:  {x.iron_oxide:.3f} g/L")
    print(f"  TM_Spirulina:{x.TM_spirulina:.4f}")
    print(f"  TM_Urine:    {x.TM_urine:.4f}")
    print(f"  TM_Winery:   {x.TM_winery:.4f}")
    print(f"  TM_Molasses: {x.TM_molasses:.4f}")
    print()
    print("--- CONTROL ---")
    print(f"  HRT:         {HRT:.1f} hours")
    print(f"  Phase:       {phase}")
    print()


def simulate(u_series: List[Dict[str, float]], x0: MFCState, dt_hours: float,
             params: MFCParams, add_noise: bool = True,
             print_interval: int = 1, verbose: bool = True) -> Dict[str, List]:
    n_steps = len(u_series)

    results = {
        'EC': [], 'pH': [], 'iron_oxide': [],
        'TM_spirulina': [], 'TM_urine': [], 'TM_winery': [], 'TM_molasses': [],
        'TM_total_other': [], 'HRT': [], 'phase': [],
        'ORP_obs': [], 'V_obs': [], 'ORP_true': [], 'V_true': [],
        'time_minutes': [],
    }

    cumulative_dosing = {
        'spirulina': 0.0,
        'urine': 0.0,
        'winery_ww': 0.0,
        'iron_oxide': 0.0,
        'molasses': 0.0,
    }

    x = x0

    for k in range(n_steps):
        u = u_series[k]

        for key in cumulative_dosing:
            cumulative_dosing[key] += u.get(key, 0.0)

        x, HRT = update_states(x, u, dt_hours, params)
        (ORP_obs, V_obs), (ORP_true, V_true) = compute_outputs(x, params, add_noise=add_noise)
        phase = get_phase(x.pH, params)

        results['EC'].append(x.EC)
        results['pH'].append(x.pH)
        results['iron_oxide'].append(x.iron_oxide)
        results['TM_spirulina'].append(x.TM_spirulina)
        results['TM_urine'].append(x.TM_urine)
        results['TM_winery'].append(x.TM_winery)
        results['TM_molasses'].append(x.TM_molasses)
        results['TM_total_other'].append(x.total_other_TM())
        results['HRT'].append(HRT)
        results['phase'].append(phase)
        results['ORP_obs'].append(ORP_obs)
        results['V_obs'].append(V_obs)
        results['ORP_true'].append(ORP_true)
        results['V_true'].append(V_true)
        results['time_minutes'].append(k)

        if verbose and (k % print_interval == 0):
            print_arduino_output(k, u, x, ORP_obs, V_obs, HRT, phase)

    results['cumulative_dosing'] = cumulative_dosing
    return results


def export_results_to_csv(results: dict, filename: str = "mfc_simulated_data.csv") -> None:
    fieldnames = [
        "time_minutes", "EC", "pH", "iron_oxide",
        "TM_spirulina", "TM_urine", "TM_winery", "TM_molasses", "TM_total_other",
        "HRT", "phase",
        "ORP_obs", "V_obs", "ORP_true", "V_true"
    ]

    with open(filename, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        n = len(results["EC"])
        for i in range(n):
            writer.writerow({
                "time_minutes": results["time_minutes"][i],
                "EC": results["EC"][i],
                "pH": results["pH"][i],
                "iron_oxide": results["iron_oxide"][i],
                "TM_spirulina": results["TM_spirulina"][i],
                "TM_urine": results["TM_urine"][i],
                "TM_winery": results["TM_winery"][i],
                "TM_molasses": results["TM_molasses"][i],
                "TM_total_other": results["TM_total_other"][i],
                "HRT": results["HRT"][i],
                "phase": results["phase"][i],
                "ORP_obs": results["ORP_obs"][i],
                "V_obs": results["V_obs"][i],
                "ORP_true": results["ORP_true"][i],
                "V_true": results["V_true"][i],
            })

    print(f"Export complete: {filename}")


def print_final_summary(results: dict, total_minutes: int) -> None:
    total_hours = total_minutes / 60.0

    print("\n")
    print("=" * 70)
    print("                     FINAL SIMULATION SUMMARY")
    print("=" * 70)
    print(f"  Total Simulation Time: {total_minutes} minutes ({total_hours:.1f} hours)")
    print("=" * 70)

    print("\n┌─────────────────────────────────────────────────────────────────────┐")
    print("│                        FINAL SENSOR OUTPUTS                         │")
    print("├─────────────────────────────────────────────────────────────────────┤")
    print(f"│  EC (Electrical Conductivity)    │  {results['EC'][-1]:>12.4f} mS/cm          │")
    print(f"│  pH                              │  {results['pH'][-1]:>12.4f}                │")
    print(f"│  ORP (Oxidation-Reduction)       │  {results['ORP_obs'][-1]:>12.2f} mV            │")
    print(f"│  Voltage                         │  {results['V_obs'][-1]:>12.4f} V             │")
    print(f"│  HRT (Hydraulic Retention Time)  │  {results['HRT'][-1]:>12.1f} hours          │")
    print(f"│  Operating Phase                 │  {results['phase'][-1]:>12}               │")
    print("└─────────────────────────────────────────────────────────────────────┘")

    print("\n┌─────────────────────────────────────────────────────────────────────┐")
    print("│                      FINAL TRACE METAL LEVELS                       │")
    print("├─────────────────────────────────────────────────────────────────────┤")
    print(f"│  Iron Oxide (Fe3O4)              │  {results['iron_oxide'][-1]:>12.4f} g/L           │")
    print(f"│  TM from Spirulina               │  {results['TM_spirulina'][-1]:>12.4f}               │")
    print(f"│  TM from Urine                   │  {results['TM_urine'][-1]:>12.4f}               │")
    print(f"│  TM from Winery WW               │  {results['TM_winery'][-1]:>12.4f}               │")
    print(f"│  TM from Molasses                │  {results['TM_molasses'][-1]:>12.4f}               │")
    print(f"│  Total Other TM                  │  {results['TM_total_other'][-1]:>12.4f}               │")
    print("└─────────────────────────────────────────────────────────────────────┘")

    cumulative = results.get('cumulative_dosing', {})

    print("\n┌─────────────────────────────────────────────────────────────────────┐")
    print("│                    TOTAL INPUTS USED (CUMULATIVE)                   │")
    print("├─────────────────────────────────────────────────────────────────────┤")
    print(f"│  Spirulina                       │  {cumulative.get('spirulina', 0):>12.2f} units         │")
    print(f"│  Urine                           │  {cumulative.get('urine', 0):>12.2f} units         │")
    print(f"│  Winery Wastewater               │  {cumulative.get('winery_ww', 0):>12.2f} units         │")
    print(f"│  Iron Oxide                      │  {cumulative.get('iron_oxide', 0):>12.2f} units         │")
    print(f"│  Molasses                        │  {cumulative.get('molasses', 0):>12.2f} units         │")
    print("├─────────────────────────────────────────────────────────────────────┤")
    total_inputs = sum(cumulative.values())
    print(f"│  TOTAL ALL INPUTS                │  {total_inputs:>12.2f} units         │")
    print("└─────────────────────────────────────────────────────────────────────┘")

    print("\n┌─────────────────────────────────────────────────────────────────────┐")
    print("│                      AVERAGE DOSING RATES                           │")
    print("├─────────────────────────────────────────────────────────────────────┤")
    print(f"│  Spirulina                       │  {cumulative.get('spirulina', 0)/total_minutes:>12.4f} units/min     │")
    print(f"│  Urine                           │  {cumulative.get('urine', 0)/total_minutes:>12.4f} units/min     │")
    print(f"│  Winery Wastewater               │  {cumulative.get('winery_ww', 0)/total_minutes:>12.4f} units/min     │")
    print(f"│  Iron Oxide                      │  {cumulative.get('iron_oxide', 0)/total_minutes:>12.4f} units/min     │")
    print(f"│  Molasses                        │  {cumulative.get('molasses', 0)/total_minutes:>12.4f} units/min     │")
    print("└─────────────────────────────────────────────────────────────────────┘")

    print("\n┌─────────────────────────────────────────────────────────────────────┐")
    print("│                         STATISTICS                                  │")
    print("├─────────────────────────────────────────────────────────────────────┤")
    print(f"│  Min EC:  {min(results['EC']):>8.4f}  │  Max EC:  {max(results['EC']):>8.4f}  │  Avg EC:  {sum(results['EC'])/len(results['EC']):>8.4f} │")
    print(f"│  Min pH:  {min(results['pH']):>8.4f}  │  Max pH:  {max(results['pH']):>8.4f}  │  Avg pH:  {sum(results['pH'])/len(results['pH']):>8.4f} │")
    print(f"│  Min ORP: {min(results['ORP_obs']):>8.2f}  │  Max ORP: {max(results['ORP_obs']):>8.2f}  │  Avg ORP: {sum(results['ORP_obs'])/len(results['ORP_obs']):>8.2f} │")
    print(f"│  Min V:   {min(results['V_obs']):>8.4f}  │  Max V:   {max(results['V_obs']):>8.4f}  │  Avg V:   {sum(results['V_obs'])/len(results['V_obs']):>8.4f} │")
    print("└─────────────────────────────────────────────────────────────────────┘")
    print()


# =====================================================================
# 6. MAIN: RUN SIMULATION, EXPORT, AND PYMC BAYESIAN MODEL
# =====================================================================

if __name__ == "__main__":
    params = MFCParams()

    dt_hours = 1.0 / 60.0
    n_steps = 240  # 4 hours to give some variation
    print_every = 30

    u_series: List[Dict[str, float]] = []
    for k in range(n_steps):
        hour = k * dt_hours

        # Make TM-related inputs vary over time so TM effects are estimable
        spirulina_in = 0.5 + 0.5 * math.sin(2 * math.pi * hour / 4.0)
        urine_in = 0.3 + 0.2 * math.cos(2 * math.pi * hour / 6.0)
        winery_in = 0.6 + 0.3 * math.sin(2 * math.pi * hour / 8.0)
        molasses_in = 0.4 + 0.3 * math.cos(2 * math.pi * hour / 5.0)
        iron_in = 0.5 + 0.3 * math.sin(2 * math.pi * hour / 10.0)

        u_series.append({
            'spirulina':  max(0.0, spirulina_in),
            'urine':      max(0.0, urine_in),
            'winery_ww':  max(0.0, winery_in),
            'iron_oxide': max(0.0, iron_in),
            'molasses':   max(0.0, molasses_in),
            'T_actual_F': 70.0 + 5.0 * math.sin(2 * math.pi * hour / 24.0),
        })

    x0 = MFCState(
        EC=5.0,
        pH=7.0,
        iron_oxide=3.0,
        TM_spirulina=0.5,
        TM_urine=0.2,
        TM_winery=0.4,
        TM_molasses=0.2
    )

    print("\n" + "=" * 70)
    print("        MFC SURROGATE MODEL - ARDUINO SERIAL MONITOR OUTPUT")
    print("=" * 70)
    print(f"Starting simulation: {n_steps} minutes, printing every {print_every} minutes")
    print("=" * 70 + "\n")

    results = simulate(
        u_series=u_series,
        x0=x0,
        dt_hours=dt_hours,
        params=params,
        add_noise=True,
        print_interval=print_every,
        verbose=True
    )

    print_final_summary(results, n_steps)
    export_results_to_csv(results, "mfc_data.csv")
    print("Data exported to mfc_data.csv")

    df = pd.read_csv("mfc_data.csv")

    EC = df["EC"].values
    pH = df["pH"].values
    iron_oxide = df["iron_oxide"].values
    TM_spirulina = df["TM_spirulina"].values
    TM_urine = df["TM_urine"].values
    TM_winery = df["TM_winery"].values
    TM_molasses = df["TM_molasses"].values
    TM_total_other = df["TM_total_other"].values

    ORP_obs = df["ORP_obs"].values
    V_obs = df["V_obs"].values

    phase = df["phase"].values
    is_acidogenic = (phase == "acidogenic").astype(float)
    is_imbalance = (phase == "imbalance").astype(float)

    with pm.Model() as mfc_model:
        # ORP ~ f(states) + noise
        a0 = pm.Normal("a0", mu=-200, sigma=50)
        a_EC = pm.Normal("a_EC", mu=8, sigma=4)
        a_pH = pm.Normal("a_pH", mu=-40, sigma=15)
        a_iron = pm.Normal("a_iron", mu=10, sigma=4)
        a_TM_other = pm.Normal("a_TM_other", mu=8, sigma=4)
        sigma_ORP = pm.HalfNormal("sigma_ORP", sigma=10)

        ORP_mu = (
            a0 +
            a_EC * EC +
            a_pH * (pH - 7) +
            a_iron * iron_oxide +
            a_TM_other * TM_total_other
        )

        ORP_like = pm.Normal("ORP_like", mu=ORP_mu, sigma=sigma_ORP, observed=ORP_obs)

        # Voltage ~ g(states) + noise, Fe and TM both contribute
        b0 = pm.Normal("b0", mu=0.4, sigma=0.2)
        b_EC = pm.Normal("b_EC", mu=0.04, sigma=0.02)
        b_pH = pm.Normal("b_pH", mu=-0.025, sigma=0.015)
        b_iron = pm.Normal("b_iron", mu=0.02, sigma=0.01)
        b_TM_other = pm.Normal("b_TM_other", mu=0.03, sigma=0.015)
        sigma_V = pm.HalfNormal("sigma_V", sigma=0.02)

        V_mu = (
            b0 +
            b_EC * np.log(EC + 1e-6) +
            b_pH * (pH - 7) +
            b_iron * iron_oxide +
            b_TM_other * TM_total_other
        )

        V_like = pm.Normal("V_like", mu=V_mu, sigma=sigma_V, observed=V_obs)

        # Optional EC–pH coupling parameters (not directly used in likelihood)
        ec_ph_slope_acid = pm.Normal("ec_ph_slope_acid", mu=3.18, sigma=0.5)
        ec_ph_intercept_acid = pm.Normal("ec_ph_intercept_acid", mu=-17.46, sigma=2)
        ec_ph_slope_imbal = pm.Normal("ec_ph_slope_imbal", mu=3.47, sigma=0.5)
        ec_ph_intercept_imbal = pm.Normal("ec_ph_intercept_imbal", mu=-24.01, sigma=2)

        trace = pm.sample(1000, tune=1000, target_accept=0.9, cores=2)

    print("\n" + "=" * 60)
    print("PyMC Model Summary")
    print("=" * 60)
    summary = az.summary(trace)
    print(summary)
    summary.to_csv("pymc_summary.csv")
    print("PyMC summary saved to pymc_summary.csv")


        MFC SURROGATE MODEL - ARDUINO SERIAL MONITOR OUTPUT
Starting simulation: 240 minutes, printing every 30 minutes

[00:00] MFC Serial Monitor - Minute 0
--- DOSING (this minute) ---
  Spirulina:   0.500 units
  Urine:       0.500 units
  Winery WW:   0.600 units
  Iron Oxide:  0.500 units
  Molasses:    0.700 units
  Temperature: 70.0 F

--- SENSOR READINGS ---
  EC:          4.06 mS/cm
  pH:          7.06
  ORP:         -136.6 mV
  Voltage:     0.5324 V

--- TRACE METALS ---
  Iron Oxide:  3.000 g/L
  TM_Spirulina:0.2000
  TM_Urine:    0.1000
  TM_Winery:   0.1800
  TM_Molasses: 0.0700

--- CONTROL ---
  HRT:         24.0 hours
  Phase:       acidogenic

[00:30] MFC Serial Monitor - Minute 30
--- DOSING (this minute) ---
  Spirulina:   0.854 units
  Urine:       0.473 units
  Winery WW:   0.715 units
  Iron Oxide:  0.593 units
  Molasses:    0.643 units
  Temperature: 70.7 F

--- SENSOR READINGS ---
  EC:          4.14 mS/cm
  pH:          7.06
  ORP:         -129.9 mV
  Voltag

Output()


PyMC Model Summary
                          mean      sd   hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
a0                    -201.979  12.857 -225.849 -178.580      0.331    0.287   
a_EC                     8.610   3.033    3.141   14.379      0.077    0.071   
a_TM_other               4.976   2.500    0.423    9.679      0.059    0.058   
a_iron                  10.316   1.654    6.909   13.191      0.036    0.036   
a_pH                   -32.975  12.328  -56.124  -10.336      0.264    0.270   
b0                       0.405   0.024    0.360    0.448      0.001    0.001   
b_EC                     0.039   0.018    0.007    0.072      0.001    0.000   
b_TM_other               0.031   0.004    0.023    0.039      0.000    0.000   
b_iron                   0.019   0.002    0.014    0.023      0.000    0.000   
b_pH                    -0.027   0.014   -0.051   -0.001      0.000    0.000   
ec_ph_intercept_acid   -17.431   1.925  -21.263  -14.062      0.035    0.045   
ec_ph_intercept_imba

In [2]:
# Model V2 - 3 Input Configuration with 1L CSTR
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple
import math
import random
import csv

import pymc as pm
import pandas as pd
import numpy as np
import arviz as az


# =====================================================================
# 1. PARAMETERS AND STATE DEFINITIONS
# =====================================================================

@dataclass
class InputRecipes:
    """
    Defines the composition of each input solution.

    INPUT 1: Pure Fermented Urine (100%)
    - No dilution, straight fermented urine
    - Rich in nitrogen, potassium, phosphorus
    - Alkaline (pH ~9 after fermentation)
    - High EC due to dissolved salts

    INPUT 2: Wine/Molasses/Iron Oxide Mix (per liter of solution)
    - Winery wastewater: carbon source, organic acids
    - Molasses: easily fermentable sugars, trace minerals
    - Iron oxide: electron acceptor, maintains 2-4 g/L in reactor
    - Iron oxide concentration set high (10 g/L) because this input
      is only ~30-40% of total flow, so need 3x concentration to
      achieve 3 g/L target in reactor

    INPUT 3: Spirulina Solution (per liter of solution)
    - Spirulina powder: protein, trace metals, vitamins
    - Provides diverse micronutrients for microbial community
    """

    # Input 2: Wine/Molasses/Iron Oxide Mix (per liter of solution)
    input2_winery_ww_mL_per_L: float = 500.0       # 500 mL winery WW per L
    input2_molasses_g_per_L: float = 50.0          # 50 g molasses per L
    input2_iron_oxide_g_per_L: float = 10.0        # 10 g/L iron oxide (high to achieve 2-4 g/L in reactor)
    input2_distilled_water_mL_per_L: float = 500.0 # 500 mL distilled water per L

    # Input 3: Spirulina Solution (per liter of solution)
    input3_spirulina_g_per_L: float = 20.0         # 20 g spirulina powder per L
    input3_distilled_water_mL_per_L: float = 1000.0


@dataclass
class MFCParams:
    # --- Reactor configuration (CSTR) ---
    reactor_volume_mL: float = 1000.0  # 1 liter working volume, constant

    # --- HRT control parameters (pH-driven) ---
    HRT_nom: float = 24.0   # nominal HRT in hours
    HRT_min: float = 6.0    # minimum HRT (fastest flow)
    HRT_max: float = 72.0   # maximum HRT (slowest flow)
    K_low_pH: float = 4.0
    K_high_pH: float = 4.0

    pH_low: float = 6.8
    pH_high: float = 7.2
    pH_low_action: float = 6.0
    pH_high_action: float = 8.0

    # --- pH decay rate ---
    pH_decay_per_24h: float = 0.4  # pH units lost per 24 hours due to fermentation

    # --- EC–pH coupling thresholds ---
    pH_acidogenic_max: float = 7.5
    pH_methanogenic_min: float = 7.5
    pH_methanogenic_max: float = 8.0
    pH_imbalance_min: float = 8.3

    ec_ph_acidogenic_slope: float = 3.18
    ec_ph_acidogenic_intercept: float = -17.46
    ec_ph_imbalance_slope: float = 3.47
    ec_ph_imbalance_intercept: float = -24.01

    # --- Iron oxide target range in reactor ---
    iron_oxide_min: float = 2.0   # g/L
    iron_oxide_max: float = 4.0   # g/L
    iron_oxide_target: float = 3.0

    # --- Characteristic concentrations for each input's effect on reactor state ---
    # These define the "strength" of each component when added to reactor

    # EC contributions (mS/cm per g/L or per mL/L in reactor)
    EC_per_urine_mL_per_L: float = 0.015          # mS/cm per mL/L of urine
    EC_per_winery_mL_per_L: float = 0.010         # mS/cm per mL/L of winery WW
    EC_per_molasses_g_per_L: float = 0.08         # mS/cm per g/L of molasses
    EC_per_iron_g_per_L: float = 0.25             # mS/cm per g/L of iron oxide
    EC_per_spirulina_g_per_L: float = 0.12        # mS/cm per g/L of spirulina

    # pH effects (deviation from neutral per concentration unit)
    pH_per_urine_mL_per_L: float = 0.003          # pH increase per mL/L urine (alkaline)
    pH_per_winery_mL_per_L: float = -0.002        # pH decrease per mL/L winery (acidic)
    pH_per_molasses_g_per_L: float = -0.015       # pH decrease per g/L molasses (ferments to acid)
    pH_per_iron_g_per_L: float = -0.05            # pH decrease per g/L iron oxide
    pH_per_spirulina_g_per_L: float = 0.02        # pH increase per g/L spirulina (slightly alkaline)

    # Trace metal contributions (arbitrary units per concentration)
    TM_per_urine_mL_per_L: float = 0.002
    TM_per_winery_mL_per_L: float = 0.003
    TM_per_molasses_g_per_L: float = 0.02
    TM_per_spirulina_g_per_L: float = 0.05

    EC_min: float = 0.2
    EC_max: float = 25.0
    pH_min: float = 4.5
    pH_max: float = 9.5
    TM_min: float = 0.0
    TM_max: float = 10.0

    # --- ORP model parameters ---
    a0_ORP: float = -200.0
    a_EC_ORP: float = 8.0
    a_pH_ORP: float = -40.0
    a_iron_ORP: float = 10.0
    a_TM_ORP: float = 8.0

    # --- Voltage model parameters ---
    E0: float = 0.4
    b_EC_V: float = 0.04
    b_pH_V: float = -0.025
    b_iron_V: float = 0.02
    b_TM_V: float = 0.03

    EC_ref_for_log: float = 1.0

    sigma_ORP: float = 4.0
    sigma_V: float = 0.005


@dataclass
class MFCState:
    """
    State variables representing concentrations IN THE REACTOR.
    All concentrations are per liter of reactor volume.
    """
    EC: float              # mS/cm - electrical conductivity
    pH: float              # pH units
    iron_oxide_g_L: float  # g/L of iron oxide in reactor
    TM_total: float        # arbitrary units - total trace metals

    # Track component concentrations for mass balance
    urine_mL_L: float      # effective mL of urine per L reactor
    winery_mL_L: float     # effective mL of winery WW per L reactor
    molasses_g_L: float    # g/L of molasses in reactor
    spirulina_g_L: float   # g/L of spirulina in reactor


@dataclass
class InputFlows:
    """Records the actual mL of each input used per time step"""
    input1_urine_mL: float
    input2_mix_mL: float
    input3_spirulina_mL: float
    total_mL: float


# =====================================================================
# 2. FLOW CALCULATIONS AND INPUT RATIOS
# =====================================================================

def calculate_flow_rate_mL_per_min(HRT_hours: float, reactor_volume_mL: float) -> float:
    """
    Calculate total flow rate in mL/min based on HRT.
    HRT = V / Q, so Q = V / HRT
    """
    flow_rate_mL_per_hour = reactor_volume_mL / HRT_hours
    flow_rate_mL_per_min = flow_rate_mL_per_hour / 60.0
    return flow_rate_mL_per_min


def determine_input_ratios(x: MFCState, params: MFCParams) -> Tuple[float, float, float]:
    """
    Determine the ratio of each input based on current reactor state.
    Returns (ratio_input1, ratio_input2, ratio_input3) that sum to 1.0

    Strategy:
    - If pH is low (acidic): increase urine (alkaline) and spirulina, decrease mix
    - If pH is high (alkaline): increase mix (acidic components), decrease urine
    - If iron is low: increase mix (contains iron oxide)
    - If iron is high: decrease mix
    """
    # Base ratios (roughly equal)
    r1 = 0.35  # urine
    r2 = 0.35  # mix (wine/molasses/iron)
    r3 = 0.30  # spirulina

    # Adjust based on pH
    if x.pH < params.pH_low:
        # Too acidic - add more alkaline inputs
        r1 += 0.15  # more urine (alkaline)
        r3 += 0.05  # more spirulina (slightly alkaline)
        r2 -= 0.20  # less acidic mix
    elif x.pH > params.pH_high:
        # Too alkaline - add more acidic inputs
        r2 += 0.15  # more mix (acidic)
        r1 -= 0.15  # less urine

    # Adjust based on iron oxide level
    if x.iron_oxide_g_L < params.iron_oxide_min:
        # Need more iron
        r2 += 0.10
        r1 -= 0.05
        r3 -= 0.05
    elif x.iron_oxide_g_L > params.iron_oxide_max:
        # Too much iron
        r2 -= 0.10
        r1 += 0.05
        r3 += 0.05

    # Ensure ratios are valid
    r1 = max(0.1, min(0.6, r1))
    r2 = max(0.1, min(0.6, r2))
    r3 = max(0.1, min(0.6, r3))

    # Normalize to sum to 1
    total = r1 + r2 + r3
    return (r1/total, r2/total, r3/total)


def calculate_input_flows(
    total_flow_mL: float,
    ratios: Tuple[float, float, float]
) -> InputFlows:
    """Calculate actual mL of each input for this time step"""
    r1, r2, r3 = ratios
    return InputFlows(
        input1_urine_mL=total_flow_mL * r1,
        input2_mix_mL=total_flow_mL * r2,
        input3_spirulina_mL=total_flow_mL * r3,
        total_mL=total_flow_mL
    )


def calculate_inflow_concentrations(
    flows: InputFlows,
    recipes: InputRecipes
) -> Dict[str, float]:
    """
    Calculate the concentrations of each component in the combined inflow.
    Returns concentrations as if the total inflow were 1 liter.
    """
    total_mL = flows.total_mL
    if total_mL <= 0:
        return {
            'urine_mL_L': 0, 'winery_mL_L': 0, 'molasses_g_L': 0,
            'iron_oxide_g_L': 0, 'spirulina_g_L': 0
        }

    # Scale factor to convert to "per liter of inflow"
    scale = 1000.0 / total_mL

    # Input 1: Pure urine
    urine_mL = flows.input1_urine_mL * scale  # mL urine per L of total inflow

    # Input 2: Mix - calculate component amounts
    input2_L = flows.input2_mix_mL / 1000.0  # convert to L
    winery_mL = (input2_L * recipes.input2_winery_ww_mL_per_L) * scale
    molasses_g = (input2_L * recipes.input2_molasses_g_per_L) * scale
    iron_oxide_g = (input2_L * recipes.input2_iron_oxide_g_per_L) * scale

    # Input 3: Spirulina solution
    input3_L = flows.input3_spirulina_mL / 1000.0
    spirulina_g = (input3_L * recipes.input3_spirulina_g_per_L) * scale

    return {
        'urine_mL_L': urine_mL,
        'winery_mL_L': winery_mL,
        'molasses_g_L': molasses_g,
        'iron_oxide_g_L': iron_oxide_g,
        'spirulina_g_L': spirulina_g
    }


# =====================================================================
# 3. CONTROL LAW
# =====================================================================

def hrt_control(pH: float, params: MFCParams) -> float:
    """Adjust HRT based on pH to maintain optimal conditions"""
    if params.pH_low <= pH <= params.pH_high:
        return params.HRT_nom

    if pH < params.pH_low_action:
        delta_pH = params.pH_low - pH
        HRT = params.HRT_nom + params.K_low_pH * delta_pH
        return max(params.HRT_min, min(params.HRT_max, HRT))

    if pH > params.pH_high_action:
        delta_pH = pH - params.pH_high
        HRT = params.HRT_nom - params.K_high_pH * delta_pH
        return max(params.HRT_min, min(params.HRT_max, HRT))

    return params.HRT_nom


def get_phase(pH: float, params: MFCParams) -> str:
    if pH < params.pH_acidogenic_max:
        return "acidogenic"
    elif params.pH_methanogenic_min <= pH <= params.pH_methanogenic_max:
        return "methanogenic"
    elif pH > params.pH_imbalance_min:
        return "imbalance"
    else:
        return "transition"


# =====================================================================
# 4. CSTR STATE UPDATE (MASS BALANCE)
# =====================================================================

def update_state_cstr(
    x: MFCState,
    flows: InputFlows,
    inflow_conc: Dict[str, float],
    dt_min: float,
    params: MFCParams,
    recipes: InputRecipes
) -> MFCState:
    """
    Update reactor state using CSTR mass balance.

    For a CSTR: dC/dt = (Q/V) * (C_in - C) + reaction_rate

    With constant volume V and flow Q:
    - Dilution rate D = Q/V = 1/HRT
    - C_new = C_old + dt * D * (C_in - C_old) + dt * reaction_rate
    """

    # Calculate current HRT and dilution rate
    HRT_hours = hrt_control(x.pH, params)
    D_per_hour = 1.0 / HRT_hours  # dilution rate (1/hour)
    D_per_min = D_per_hour / 60.0  # dilution rate (1/min)

    # Fraction of reactor volume replaced this time step
    dilution_fraction = D_per_min * dt_min
    dilution_fraction = min(1.0, dilution_fraction)  # cap at 100%

    # Update component concentrations via CSTR mass balance
    # C_new = C_old * (1 - D*dt) + C_in * D*dt

    urine_new = x.urine_mL_L * (1 - dilution_fraction) + inflow_conc['urine_mL_L'] * dilution_fraction
    winery_new = x.winery_mL_L * (1 - dilution_fraction) + inflow_conc['winery_mL_L'] * dilution_fraction
    molasses_new = x.molasses_g_L * (1 - dilution_fraction) + inflow_conc['molasses_g_L'] * dilution_fraction
    spirulina_new = x.spirulina_g_L * (1 - dilution_fraction) + inflow_conc['spirulina_g_L'] * dilution_fraction
    iron_new = x.iron_oxide_g_L * (1 - dilution_fraction) + inflow_conc['iron_oxide_g_L'] * dilution_fraction

    # Clamp iron oxide to target range
    iron_new = max(params.iron_oxide_min * 0.5, min(params.iron_oxide_max * 1.5, iron_new))

    # Calculate derived state variables from concentrations

    # EC from all components
    EC_new = (
        params.EC_per_urine_mL_per_L * urine_new +
        params.EC_per_winery_mL_per_L * winery_new +
        params.EC_per_molasses_g_per_L * molasses_new +
        params.EC_per_iron_g_per_L * iron_new +
        params.EC_per_spirulina_g_per_L * spirulina_new
    )
    EC_new = max(params.EC_min, min(params.EC_max, EC_new))

    # pH from all components (relative to neutral)
    pH_new = 7.0 + (
        params.pH_per_urine_mL_per_L * urine_new +
        params.pH_per_winery_mL_per_L * winery_new +
        params.pH_per_molasses_g_per_L * molasses_new +
        params.pH_per_iron_g_per_L * iron_new +
        params.pH_per_spirulina_g_per_L * spirulina_new
    )

    # Apply pH decay from fermentation
    pH_decay_per_min = params.pH_decay_per_24h / (24.0 * 60.0)
    pH_new -= pH_decay_per_min * dt_min
    pH_new = max(params.pH_min, min(params.pH_max, pH_new))

    # Trace metals from components
    TM_new = (
        params.TM_per_urine_mL_per_L * urine_new +
        params.TM_per_winery_mL_per_L * winery_new +
        params.TM_per_molasses_g_per_L * molasses_new +
        params.TM_per_spirulina_g_per_L * spirulina_new
    )
    TM_new = max(params.TM_min, min(params.TM_max, TM_new))

    return MFCState(
        EC=EC_new,
        pH=pH_new,
        iron_oxide_g_L=iron_new,
        TM_total=TM_new,
        urine_mL_L=urine_new,
        winery_mL_L=winery_new,
        molasses_g_L=molasses_new,
        spirulina_g_L=spirulina_new
    )


# =====================================================================
# 5. OUTPUT MODEL: STATES → ORP AND VOLTAGE
# =====================================================================

def compute_outputs(
    x: MFCState,
    params: MFCParams,
    add_noise: bool = True
) -> Tuple[Tuple[float, float], Tuple[float, float]]:
    """Compute ORP and Voltage from reactor state"""

    ORP_true = (
        params.a0_ORP +
        params.a_EC_ORP * x.EC +
        params.a_pH_ORP * (x.pH - 7.0) +
        params.a_iron_ORP * x.iron_oxide_g_L +
        params.a_TM_ORP * x.TM_total
    )

    EC_for_log = max(1e-6, x.EC / params.EC_ref_for_log)
    V_true = (
        params.E0 +
        params.b_EC_V * math.log(EC_for_log) +
        params.b_pH_V * (x.pH - 7.0) +
        params.b_iron_V * x.iron_oxide_g_L +
        params.b_TM_V * x.TM_total
    )

    if add_noise:
        ORP_obs = ORP_true + random.gauss(0.0, params.sigma_ORP)
        V_obs = V_true + random.gauss(0.0, params.sigma_V)
    else:
        ORP_obs = ORP_true
        V_obs = V_true

    return (ORP_obs, V_obs), (ORP_true, V_true)


# =====================================================================
# 6. SIMULATION DRIVER
# =====================================================================

def print_arduino_output(
    minute: int,
    flows: InputFlows,
    x: MFCState,
    ORP: float,
    V: float,
    HRT: float,
    phase: str
) -> None:
    hours = minute // 60
    mins = minute % 60

    print("=" * 70)
    print(f"[{hours:02d}:{mins:02d}] MFC Serial Monitor - Minute {minute}")
    print("=" * 70)
    print("--- INPUT FLOWS (this minute) ---")
    print(f"  Input 1 (Urine):        {flows.input1_urine_mL:>8.3f} mL")
    print(f"  Input 2 (Wine/Mol/Fe):  {flows.input2_mix_mL:>8.3f} mL")
    print(f"  Input 3 (Spirulina):    {flows.input3_spirulina_mL:>8.3f} mL")
    print(f"  TOTAL FLOW:             {flows.total_mL:>8.3f} mL")
    print()
    print("--- REACTOR STATE (1L volume) ---")
    print(f"  EC:          {x.EC:.2f} mS/cm")
    print(f"  pH:          {x.pH:.2f}")
    print(f"  Iron Oxide:  {x.iron_oxide_g_L:.3f} g/L")
    print(f"  Trace Metals:{x.TM_total:.3f} units")
    print()
    print("--- SENSOR OUTPUTS ---")
    print(f"  ORP:         {ORP:.1f} mV")
    print(f"  Voltage:     {V:.4f} V")
    print()
    print("--- CONTROL ---")
    print(f"  HRT:         {HRT:.1f} hours")
    print(f"  Flow Rate:   {flows.total_mL:.3f} mL/min")
    print(f"  Phase:       {phase}")
    print()


def simulate(
    n_steps: int,
    x0: MFCState,
    dt_min: float,
    params: MFCParams,
    recipes: InputRecipes,
    add_noise: bool = True,
    print_interval: int = 1,
    verbose: bool = True
) -> Dict[str, List]:

    results = {
        'time_minutes': [],
        'EC': [], 'pH': [], 'iron_oxide_g_L': [], 'TM_total': [],
        'urine_mL_L': [], 'winery_mL_L': [], 'molasses_g_L': [], 'spirulina_g_L': [],
        'HRT': [], 'phase': [],
        'ORP_obs': [], 'V_obs': [], 'ORP_true': [], 'V_true': [],
        'input1_mL': [], 'input2_mL': [], 'input3_mL': [], 'total_flow_mL': [],
    }

    # Cumulative tracking
    cumulative = {
        'input1_urine_mL': 0.0,
        'input2_mix_mL': 0.0,
        'input3_spirulina_mL': 0.0,
        'total_mL': 0.0,
        # Track actual feedstock amounts
        'urine_mL': 0.0,
        'winery_ww_mL': 0.0,
        'molasses_g': 0.0,
        'iron_oxide_g': 0.0,
        'spirulina_g': 0.0,
        'distilled_water_mL': 0.0,
    }

    x = x0

    for k in range(n_steps):
        # Get current HRT and flow rate
        HRT = hrt_control(x.pH, params)
        total_flow_mL = calculate_flow_rate_mL_per_min(HRT, params.reactor_volume_mL) * dt_min

        # Determine input ratios based on current state
        ratios = determine_input_ratios(x, params)

        # Calculate actual flows
        flows = calculate_input_flows(total_flow_mL, ratios)

        # Calculate inflow concentrations
        inflow_conc = calculate_inflow_concentrations(flows, recipes)

        # Update reactor state
        x = update_state_cstr(x, flows, inflow_conc, dt_min, params, recipes)

        # Compute outputs
        (ORP_obs, V_obs), (ORP_true, V_true) = compute_outputs(x, params, add_noise)
        phase = get_phase(x.pH, params)

        # Store results
        results['time_minutes'].append(k)
        results['EC'].append(x.EC)
        results['pH'].append(x.pH)
        results['iron_oxide_g_L'].append(x.iron_oxide_g_L)
        results['TM_total'].append(x.TM_total)
        results['urine_mL_L'].append(x.urine_mL_L)
        results['winery_mL_L'].append(x.winery_mL_L)
        results['molasses_g_L'].append(x.molasses_g_L)
        results['spirulina_g_L'].append(x.spirulina_g_L)
        results['HRT'].append(HRT)
        results['phase'].append(phase)
        results['ORP_obs'].append(ORP_obs)
        results['V_obs'].append(V_obs)
        results['ORP_true'].append(ORP_true)
        results['V_true'].append(V_true)
        results['input1_mL'].append(flows.input1_urine_mL)
        results['input2_mL'].append(flows.input2_mix_mL)
        results['input3_mL'].append(flows.input3_spirulina_mL)
        results['total_flow_mL'].append(flows.total_mL)

        # Update cumulative totals
        cumulative['input1_urine_mL'] += flows.input1_urine_mL
        cumulative['input2_mix_mL'] += flows.input2_mix_mL
        cumulative['input3_spirulina_mL'] += flows.input3_spirulina_mL
        cumulative['total_mL'] += flows.total_mL

        # Track actual feedstock usage
        cumulative['urine_mL'] += flows.input1_urine_mL

        input2_L = flows.input2_mix_mL / 1000.0
        cumulative['winery_ww_mL'] += input2_L * recipes.input2_winery_ww_mL_per_L
        cumulative['molasses_g'] += input2_L * recipes.input2_molasses_g_per_L
        cumulative['iron_oxide_g'] += input2_L * recipes.input2_iron_oxide_g_per_L
        cumulative['distilled_water_mL'] += input2_L * recipes.input2_distilled_water_mL_per_L

        input3_L = flows.input3_spirulina_mL / 1000.0
        cumulative['spirulina_g'] += input3_L * recipes.input3_spirulina_g_per_L
        cumulative['distilled_water_mL'] += input3_L * recipes.input3_distilled_water_mL_per_L

        # Print if verbose
        if verbose and (k % print_interval == 0):
            print_arduino_output(k, flows, x, ORP_obs, V_obs, HRT, phase)

    results['cumulative'] = cumulative
    return results


def export_results_to_csv(results: dict, filename: str = "mfc_data.csv") -> None:
    fieldnames = [
        "time_minutes", "EC", "pH", "iron_oxide_g_L", "TM_total",
        "urine_mL_L", "winery_mL_L", "molasses_g_L", "spirulina_g_L",
        "HRT", "phase",
        "ORP_obs", "V_obs", "ORP_true", "V_true",
        "input1_mL", "input2_mL", "input3_mL", "total_flow_mL"
    ]

    with open(filename, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        n = len(results["EC"])
        for i in range(n):
            writer.writerow({field: results[field][i] for field in fieldnames})

    print(f"Export complete: {filename}")


def print_final_summary(results: dict, total_minutes: int, recipes: InputRecipes) -> None:
    total_hours = total_minutes / 60.0
    cumulative = results.get('cumulative', {})

    print("\n")
    print("=" * 78)
    print("                         FINAL SIMULATION SUMMARY")
    print("=" * 78)
    print(f"  Total Simulation Time: {total_minutes} minutes ({total_hours:.1f} hours)")
    print(f"  Reactor Volume: 1000 mL (1 L) - Constant with overflow")
    print("=" * 78)

    print("\n┌──────────────────────────────────────────────────────────────────────────┐")
    print("│                          FINAL REACTOR STATE                             │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print(f"│  EC (Electrical Conductivity)     │  {results['EC'][-1]:>12.2f} mS/cm               │")
    print(f"│  pH                               │  {results['pH'][-1]:>12.2f}                     │")
    print(f"│  Iron Oxide                       │  {results['iron_oxide_g_L'][-1]:>12.3f} g/L                  │")
    print(f"│  Trace Metals                     │  {results['TM_total'][-1]:>12.3f} units                │")
    print(f"│  ORP                              │  {results['ORP_obs'][-1]:>12.1f} mV                   │")
    print(f"│  Voltage                          │  {results['V_obs'][-1]:>12.4f} V                    │")
    print(f"│  HRT                              │  {results['HRT'][-1]:>12.1f} hours                │")
    print(f"│  Operating Phase                  │  {results['phase'][-1]:>12}                     │")
    print("└──────────────────────────────────────────────────────────────────────────┘")

    print("\n┌──────────────────────────────────────────────────────────────────────────┐")
    print("│                    TOTAL INPUT SOLUTIONS USED                            │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print(f"│  Input 1 (Pure Urine)             │  {cumulative.get('input1_urine_mL', 0):>12.1f} mL                  │")
    print(f"│  Input 2 (Wine/Molasses/Fe Mix)   │  {cumulative.get('input2_mix_mL', 0):>12.1f} mL                  │")
    print(f"│  Input 3 (Spirulina Solution)     │  {cumulative.get('input3_spirulina_mL', 0):>12.1f} mL                  │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print(f"│  TOTAL FLUID THROUGH REACTOR      │  {cumulative.get('total_mL', 0):>12.1f} mL                  │")
    print("└──────────────────────────────────────────────────────────────────────────┘")

    print("\n┌──────────────────────────────────────────────────────────────────────────┐")
    print("│                    TOTAL RAW FEEDSTOCK CONSUMED                          │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print(f"│  Fermented Urine                  │  {cumulative.get('urine_mL', 0):>12.1f} mL                  │")
    print(f"│  Winery Wastewater                │  {cumulative.get('winery_ww_mL', 0):>12.1f} mL                  │")
    print(f"│  Molasses                         │  {cumulative.get('molasses_g', 0):>12.1f} g                   │")
    print(f"│  Iron Oxide (Fe₃O₄)               │  {cumulative.get('iron_oxide_g', 0):>12.1f} g                   │")
    print(f"│  Spirulina Powder                 │  {cumulative.get('spirulina_g', 0):>12.1f} g                   │")
    print(f"│  Distilled Water                  │  {cumulative.get('distilled_water_mL', 0):>12.1f} mL                  │")
    print("└──────────────────────────────────────────────────────────────────────────┘")

    print("\n┌──────────────────────────────────────────────────────────────────────────┐")
    print("│                         INPUT RECIPES                                    │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print("│  INPUT 1: Pure Fermented Urine                                           │")
    print("│    • 100% fermented human urine (aged 2+ weeks)                          │")
    print("│    • No dilution or additives                                            │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print("│  INPUT 2: Wine/Molasses/Iron Oxide Mix (per 1 L of solution)             │")
    print(f"│    • Winery Wastewater:    {recipes.input2_winery_ww_mL_per_L:>6.0f} mL                                │")
    print(f"│    • Molasses:             {recipes.input2_molasses_g_per_L:>6.0f} g                                 │")
    print(f"│    • Iron Oxide (Fe₃O₄):   {recipes.input2_iron_oxide_g_per_L:>6.0f} g                                 │")
    print(f"│    • Distilled Water:      {recipes.input2_distilled_water_mL_per_L:>6.0f} mL                                │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print("│  INPUT 3: Spirulina Solution (per 1 L of solution)                       │")
    print(f"│    • Spirulina Powder:     {recipes.input3_spirulina_g_per_L:>6.0f} g                                 │")
    print(f"│    • Distilled Water:      {recipes.input3_distilled_water_mL_per_L:>6.0f} mL                                │")
    print("└──────────────────────────────────────────────────────────────────────────┘")

    print("\n┌──────────────────────────────────────────────────────────────────────────┐")
    print("│                           STATISTICS                                     │")
    print("├──────────────────────────────────────────────────────────────────────────┤")
    print(f"│  EC:   Min {min(results['EC']):>6.2f}  │  Max {max(results['EC']):>6.2f}  │  Avg {sum(results['EC'])/len(results['EC']):>6.2f} mS/cm     │")
    print(f"│  pH:   Min {min(results['pH']):>6.2f}  │  Max {max(results['pH']):>6.2f}  │  Avg {sum(results['pH'])/len(results['pH']):>6.2f}           │")
    print(f"│  Fe:   Min {min(results['iron_oxide_g_L']):>6.2f}  │  Max {max(results['iron_oxide_g_L']):>6.2f}  │  Avg {sum(results['iron_oxide_g_L'])/len(results['iron_oxide_g_L']):>6.2f} g/L       │")
    print(f"│  ORP:  Min {min(results['ORP_obs']):>6.1f}  │  Max {max(results['ORP_obs']):>6.1f}  │  Avg {sum(results['ORP_obs'])/len(results['ORP_obs']):>6.1f} mV        │")
    print(f"│  V:    Min {min(results['V_obs']):>6.3f}  │  Max {max(results['V_obs']):>6.3f}  │  Avg {sum(results['V_obs'])/len(results['V_obs']):>6.3f} V          │")
    print(f"│  Flow: Min {min(results['total_flow_mL']):>6.3f}  │  Max {max(results['total_flow_mL']):>6.3f}  │  Avg {sum(results['total_flow_mL'])/len(results['total_flow_mL']):>6.3f} mL/min    │")
    print("└──────────────────────────────────────────────────────────────────────────┘")
    print()


# =====================================================================
# 7. MAIN: RUN SIMULATION
# =====================================================================

if __name__ == "__main__":
    params = MFCParams()
    recipes = InputRecipes()

    dt_min = 1.0          # 1 minute time step
    n_steps = 240         # 4 hours simulation
    print_every = 30      # print every 30 minutes

    # Initial reactor state (1L volume)
    x0 = MFCState(
        EC=5.0,
        pH=7.0,
        iron_oxide_g_L=3.0,
        TM_total=1.0,
        urine_mL_L=200.0,
        winery_mL_L=150.0,
        molasses_g_L=15.0,
        spirulina_g_L=5.0
    )

    print("\n" + "=" * 78)
    print("     MFC SURROGATE MODEL - 3 INPUT CONFIGURATION (1L CSTR)")
    print("=" * 78)
    print(f"Starting simulation: {n_steps} minutes, printing every {print_every} minutes")
    print("=" * 78 + "\n")

    results = simulate(
        n_steps=n_steps,
        x0=x0,
        dt_min=dt_min,
        params=params,
        recipes=recipes,
        add_noise=True,
        print_interval=print_every,
        verbose=True
    )

    print_final_summary(results, n_steps, recipes)
    export_results_to_csv(results, "mfc_data.csv")
    print("Data exported to mfc_data.csv")

    # =====================================================================
    # PYMC BAYESIAN MODEL
    # =====================================================================

    df = pd.read_csv("mfc_data.csv")

    EC = df["EC"].values
    pH = df["pH"].values
    iron_oxide = df["iron_oxide_g_L"].values
    TM_total = df["TM_total"].values
    ORP_obs = df["ORP_obs"].values
    V_obs = df["V_obs"].values

    with pm.Model() as mfc_model:
        # ORP model
        a0 = pm.Normal("a0", mu=-200, sigma=50)
        a_EC = pm.Normal("a_EC", mu=8, sigma=4)
        a_pH = pm.Normal("a_pH", mu=-40, sigma=15)
        a_iron = pm.Normal("a_iron", mu=10, sigma=4)
        a_TM = pm.Normal("a_TM", mu=8, sigma=4)
        sigma_ORP = pm.HalfNormal("sigma_ORP", sigma=10)

        ORP_mu = (
            a0 +
            a_EC * EC +
            a_pH * (pH - 7) +
            a_iron * iron_oxide +
            a_TM * TM_total
        )
        ORP_like = pm.Normal("ORP_like", mu=ORP_mu, sigma=sigma_ORP, observed=ORP_obs)

        # Voltage model
        b0 = pm.Normal("b0", mu=0.4, sigma=0.2)
        b_EC = pm.Normal("b_EC", mu=0.04, sigma=0.02)
        b_pH = pm.Normal("b_pH", mu=-0.025, sigma=0.015)
        b_iron = pm.Normal("b_iron", mu=0.02, sigma=0.01)
        b_TM = pm.Normal("b_TM", mu=0.03, sigma=0.015)
        sigma_V = pm.HalfNormal("sigma_V", sigma=0.02)

        V_mu = (
            b0 +
            b_EC * np.log(EC + 1e-6) +
            b_pH * (pH - 7) +
            b_iron * iron_oxide +
            b_TM * TM_total
        )
        V_like = pm.Normal("V_like", mu=V_mu, sigma=sigma_V, observed=V_obs)

        trace = pm.sample(1000, tune=1000, target_accept=0.9, cores=2)

    print("\n" + "=" * 60)
    print("PyMC Model Summary")
    print("=" * 60)
    summary = az.summary(trace)
    print(summary)
    summary.to_csv("pymc_summary.csv")
    print("PyMC summary saved to pymc_summary.csv")


     MFC SURROGATE MODEL - 3 INPUT CONFIGURATION (1L CSTR)
Starting simulation: 240 minutes, printing every 30 minutes

[00:00] MFC Serial Monitor - Minute 0
--- INPUT FLOWS (this minute) ---
  Input 1 (Urine):           0.243 mL
  Input 2 (Wine/Mol/Fe):     0.243 mL
  Input 3 (Spirulina):       0.208 mL
  TOTAL FLOW:                0.694 mL

--- REACTOR STATE (1L volume) ---
  EC:          7.05 mS/cm
  pH:          7.02
  Iron Oxide:  3.000 g/L
  Trace Metals:1.400 units

--- SENSOR OUTPUTS ---
  ORP:         -103.1 mV
  Voltage:     0.5836 V

--- CONTROL ---
  HRT:         24.0 hours
  Flow Rate:   0.694 mL/min
  Phase:       acidogenic

[00:30] MFC Serial Monitor - Minute 30
--- INPUT FLOWS (this minute) ---
  Input 1 (Urine):           0.243 mL
  Input 2 (Wine/Mol/Fe):     0.243 mL
  Input 3 (Spirulina):       0.208 mL
  TOTAL FLOW:                0.694 mL

--- REACTOR STATE (1L volume) ---
  EC:          7.11 mS/cm
  pH:          7.03
  Iron Oxide:  3.011 g/L
  Trace Metals:1.410

Output()


PyMC Model Summary
              mean      sd   hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
a0        -213.545  17.622 -247.270 -182.464      0.591    0.399     888.0   
a_EC         9.864   2.301    5.975   14.479      0.077    0.056     895.0   
a_TM         8.034   4.102    0.576   15.935      0.084    0.107    2372.0   
a_iron      10.002   3.987    2.084   17.201      0.093    0.090    1848.0   
a_pH       -36.580  14.187  -64.814  -11.407      0.377    0.316    1421.0   
b0           0.420   0.034    0.358    0.484      0.001    0.001    1382.0   
b_EC         0.035   0.017    0.003    0.067      0.000    0.000    1344.0   
b_TM         0.027   0.013    0.001    0.051      0.000    0.000    1833.0   
b_iron       0.019   0.010    0.002    0.038      0.000    0.000    1777.0   
b_pH        -0.028   0.013   -0.052   -0.003      0.000    0.000    1916.0   
sigma_ORP    4.073   0.176    3.751    4.415      0.004    0.004    1857.0   
sigma_V      0.005   0.000    0.005    0.005